<a href="https://colab.research.google.com/github/Nawin03-DS/Python-Notes/blob/main/Bank_Customer_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-sql
%load_ext sql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [ ]:
%sql sqlite:///your_database.db


Question 1: Print customer Id, customer name and average account_balance maintained by each customer for all of his/her accounts in the bank. (8 Rows)

In [ ]:
SELECT
  BC.customer_id,
  BC.customer_name,
  AVG(BAD.Balance_amount) AS average_account_balance
FROM BANK_CUSTOMER AS BC
JOIN Bank_Account_Details AS BAD
  ON BC.customer_id = BAD.Customer_id
GROUP BY
  BC.customer_id,
  BC.customer_name;

Question 2: Print customer_id, account_number and balance_amount, condition that if balance_amount is nil then assign transaction_amount for account_type = "Credit Card" (4 Rows)

In [ ]:
SELECT
  BAD.Customer_id, -- Selects customer ID
  BAD.Account_Number, -- Selects account number
  CASE
    WHEN (BAD.Balance_amount IS NULL OR BAD.Balance_amount = 0) AND (BAD.Account_type = 'Credit Card' OR BAD.Account_type = 'Add-on Credit Card') -- Checks for nil balance on credit cards
    THEN (
      SELECT
        SUM(BAT.Transaction_amount) -- Sums transaction amounts if condition met
      FROM BANK_ACCOUNT_TRANSACTION AS BAT
      WHERE
        BAT.Account_Number = BAD.Account_Number
    )
    ELSE BAD.Balance_amount -- Otherwise, keeps original balance
  END AS final_balance_amount
FROM Bank_Account_Details AS BAD -- From account details table
WHERE
  BAD.Account_type IN ('Credit Card', 'Add-on Credit Card'); -- Filters for Credit Card types

Question 3: Print account_number and balance_amount, transaction_amount, Transaction_Date from Bank_Account_Details and bank_account_transaction for all the transactions occurred during march,2020 and april, 2020 (12 Rows)

In [ ]:
SELECT
  BAD.Account_Number, -- Selects account number
  BAD.Balance_amount, -- Selects balance amount
  BAT.Transaction_amount, -- Selects transaction amount
  BAT.Transaction_Date -- Selects transaction date
FROM Bank_Account_Details AS BAD -- From account details table
JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Joins with transaction table
  ON BAD.Account_Number = BAT.Account_Number -- Links by account number
WHERE
  BAT.Transaction_Date BETWEEN '2020-03-01' AND '2020-04-30' -- Filters transactions between March 1, 2020 and April 30, 2020
ORDER BY
  BAT.Transaction_Date;

Question 4: Print all of the customer id, account number, balance_amount, transaction_amount , Transaction_Date from bank_customer, Bank_Account_Details and bank_account_transaction tables where excluding all of their transactions in march, 2020 month (22 Rows)

In [ ]:
SELECT
    bad.account_number,
    bad.balance_amount,
    bat.transaction_amount,
    bat.transaction_date
FROM
    Bank_Account_Details bad
JOIN
    BANK_ACCOUNT_TRANSACTION bat
    ON bad.account_number = bat.account_number
WHERE
    strftime('%Y-%m', bat.transaction_date) IN ('2020-03', '2020-04');

Question 5: Print only the customer id, account_number, balance_amount,transaction_amount ,transaction_date who did transactions during the first quarter. Do not display the accounts if they have not done any transactions in the first quarter. (16 Rows)

In [ ]:
SELECT
  BC.customer_id, -- Selects customer ID
  BAD.Account_Number, -- Selects account number
  BAD.Balance_amount, -- Selects balance amount
  BAT.Transaction_amount, -- Selects transaction amount
  BAT.Transaction_Date -- Selects transaction date
FROM BANK_CUSTOMER AS BC -- From customer table
JOIN Bank_Account_Details AS BAD -- Joins with account details
  ON BC.customer_id = BAD.Customer_id -- Links by customer_id
JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Inner Join ensures only accounts with transactions are included
  ON BAD.Account_Number = BAT.Account_Number -- Links by account number
WHERE
  BAT.Transaction_Date BETWEEN '2020-01-01' AND '2020-03-31' -- Filters for transactions in the first quarter of 2020
ORDER BY
  BC.customer_id,
  BAT.Transaction_Date;

Question 6: Print account_number, Event and Customer_message from BANK_CUSTOMER_MESSAGES and Bank_Account_Details to display an “Adhoc" Event for all customers who have “SAVINGS" account_type account. (8 Rows)

In [ ]:
SELECT
  BAD.Account_Number, -- Selects account number
  BCM.Event, -- Selects event type
  BCM.Customer_message -- Selects customer message
FROM Bank_Account_Details AS BAD -- From account details table
CROSS JOIN BANK_CUSTOMER_MESSAGES AS BCM -- Cross joins as no direct link is specified, showing all "Adhoc" messages for all relevant accounts
WHERE
  BAD.Account_type = 'SAVINGS' AND BCM.Event = 'Adhoc'; -- Filters for SAVINGS accounts and 'Adhoc' events

Question 7: Print all Customer_id, Account_Number, Account_type, and display deducted balance_amount by subtracting only negative transaction_amounts for Relationship_type ="P" (P - means Primary , S - means Secondary ) . (27 Rows)

In [ ]:
SELECT
  BAD.Customer_id, -- Selects customer ID
  BAD.Account_Number, -- Selects account number
  BAD.Account_type, -- Selects account type
  BAD.Balance_amount AS original_balance, -- Displays original balance for comparison
  (BAD.Balance_amount + COALESCE(SUM(CASE WHEN BAT.Transaction_amount < 0 THEN BAT.Transaction_amount ELSE 0 END), 0)) AS deducted_balance_amount -- Calculates new balance by adding (subtracting negative) transaction amounts
FROM Bank_Account_Details AS BAD -- From account details table
LEFT JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Left join to include accounts even if they have no transactions
  ON BAD.Account_Number = BAT.Account_Number -- Links by account number
WHERE
  BAD.Relationship_type = 'P' -- Filters for primary relationship types
GROUP BY
  BAD.Customer_id,
  BAD.Account_Number,
  BAD.Account_type,
  BAD.Balance_amount -- Groups to apply SUM for each account
ORDER BY
  BAD.Customer_id,
  BAD.Account_Number;

Question 8: a) Display records of All Accounts, their Account_types, the transaction amount. b) Along with first step, Display other columns with corresponding linking account number, account types (15 Rows)

In [ ]:
SELECT
  BAD.Customer_id, -- Customer ID from account details
  BAD.Account_Number AS Main_Account_Number, -- Main account number
  BAD.Account_type AS Main_Account_Type, -- Main account type
  BAT.Transaction_amount, -- Transaction amount for the main account
  BAR.Linking_Account_Number, -- Linking account number from relationship details
  (
    SELECT
      Account_type -- Subquery to get the account type of the linking account
    FROM Bank_Account_Details
    WHERE
      Account_Number = BAR.Linking_Account_Number
  ) AS Linking_Account_Type
FROM Bank_Account_Details AS BAD -- From account details table
LEFT JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Left join to include accounts even if they have no transactions
  ON BAD.Account_Number = BAT.Account_Number -- Links by account number
LEFT JOIN Bank_Account_Relationship_Details AS BAR -- Left join to include accounts even if they are not linked
  ON BAD.Account_Number = BAR.Account_Number -- Links by account number
ORDER BY
  BAD.Customer_id,
  Main_Account_Number;

Question 9: a) Display records of All Accounts, their Account_types, the transaction amount. b) Along with first step, Display other columns with corresponding linking account number, account types c) After retrieving all records of accounts and their linked accounts, display the transaction amount of accounts appeared in another column. (26 Rows)

In [ ]:
SELECT
  BAD.Customer_id, -- Customer ID from account details
  BAD.Account_Number AS Main_Account_Number, -- Main account number
  BAD.Account_type AS Main_Account_Type, -- Main account type
  BAT.Transaction_amount AS Main_Account_Transaction_Amount, -- Transaction amount for the main account
  BAR.Linking_Account_Number, -- Linking account number from relationship details
  (
    SELECT
      Account_type -- Subquery to get the account type of the linking account
    FROM Bank_Account_Details
    WHERE
      Account_Number = BAR.Linking_Account_Number
  ) AS Linking_Account_Type,
  (
    SELECT
      SUM(LinkedBAT.Transaction_amount) -- Aggregate sum of transactions for the linked account
    FROM BANK_ACCOUNT_TRANSACTION AS LinkedBAT
    WHERE
      LinkedBAT.Account_Number = BAR.Linking_Account_Number
  ) AS Linked_Account_Transaction_Amount
FROM Bank_Account_Details AS BAD -- From account details table
LEFT JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Left join to include accounts even if they have no transactions
  ON BAD.Account_Number = BAT.Account_Number -- Links by account number
LEFT JOIN Bank_Account_Relationship_Details AS BAR -- Left join to include accounts even if they are not linked
  ON BAD.Account_Number = BAR.Account_Number -- Links by account number
ORDER BY
  BAD.Customer_id,
  Main_Account_Number;

Question 10: Display all saving account holders have “Add-on Credit Cards" and “Credit cards" (3 Rows)

In [ ]:
SELECT DISTINCT
  BC.customer_id, -- Selects distinct customer IDs
  BC.customer_name -- Selects customer names
FROM BANK_CUSTOMER AS BC -- From customer table
JOIN Bank_Account_Details AS BAD_SAVINGS -- Join for SAVINGS account
  ON BC.customer_id = BAD_SAVINGS.Customer_id
  AND BAD_SAVINGS.Account_type = 'SAVINGS' -- Filters for SAVINGS accounts
WHERE
  EXISTS (
    SELECT
      1
    FROM Bank_Account_Details AS BAD_CC -- Checks if the customer also has a 'Credit Card'
    WHERE
      BAD_CC.Customer_id = BC.customer_id AND BAD_CC.Account_type = 'Credit Card'
  )
  AND EXISTS (
    SELECT
      1
    FROM Bank_Account_Details AS BAD_ADDON_CC -- Checks if the customer also has an 'Add-on Credit Card'
    WHERE
      BAD_ADDON_CC.Customer_id = BC.customer_id AND BAD_ADDON_CC.Account_type = 'Add-on Credit Card'
  );


Question 11: Display records of “SAVINGS” account linked with “Credit cards" account_type and its credit aggregate sum of transaction amount. (1 Row)

In [ ]:
SELECT
  BAR.Linking_Account_Number AS Savings_Account_Number, -- The savings account number that is linked
  BAD_Savings.Account_type AS Savings_Account_Type, -- The type of the savings account
  BAR.Account_Number AS Credit_Card_Account_Number, -- The credit card account number
  BAD_CreditCard.Account_type AS Credit_Card_Account_Type, -- The type of the credit card account
  SUM(BAT.Transaction_amount) AS Total_Credit_Card_Transaction_Amount -- Sum of transactions for the linked credit card
FROM Bank_Account_Relationship_Details AS BAR -- From relationship details table
JOIN Bank_Account_Details AS BAD_Savings -- Join to get details of the SAVINGS account (the linked account)
  ON BAR.Linking_Account_Number = BAD_Savings.Account_Number
  AND BAD_Savings.Account_type = 'SAVINGS' -- Filters for SAVINGS account type
JOIN Bank_Account_Details AS BAD_CreditCard -- Join to get details of the Credit Card account (the main account in relationship)
  ON BAR.Account_Number = BAD_CreditCard.Account_Number
  AND (
    BAD_CreditCard.Account_type = 'Credit Card' OR BAD_CreditCard.Account_type = 'Add-on Credit Card'
  ) -- Filters for Credit Card types
LEFT JOIN BANK_ACCOUNT_TRANSACTION AS BAT -- Left join to get transactions for the Credit Card
  ON BAR.Account_Number = BAT.Account_Number -- Links by account number
GROUP BY
  BAR.Linking_Account_Number,
  BAD_Savings.Account_type,
  BAR.Account_Number,
  BAD_CreditCard.Account_type;